In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# Get list of behaviors

In [72]:
def get_behaviors_list(exp_dir):
    """
    Finds first JAABA experiment and returns list of all behavioral files.
    """
    flybowl_path = os.path.join(exp_dir, next(os.walk(exp_dir))[1][0])
    return list(filter(lambda x: x.startswith('scores'), os.listdir(flybowl_path)))
behaviors_list = get_behaviors_list(os.path.join('/content/drive/MyDrive/0 Flybowl/48 hrs-JAABAData/13.17.23'))
print(behaviors_list)

['scores_AttemptedCopulation.mat', 'scores_BodyTurns.mat', 'scores_Chaining.mat', 'scores_Chasev7.mat', 'scores_Copulation.mat', 'scores_Crabwalk2.mat', 'scores_Crabwalk3.mat', 'scores_Jump.mat', 'scores_pivot_center.mat', 'scores_pivot_tail.mat', 'scores_Stops.mat', 'scoresTouch.mat', 'scores_Walk.mat', 'scoresBackup.mat']


# Set up folder structure for data storage

In [ ]:
for behavior in behaviors_list:
    os.mkdir(os.path.join('/content/drive/MyDrive/0 Flybowl/test', behavior[behavior.find("_")+1:behavior.rfind(".")]))

# Get experimental groupings for each line

In [74]:
def get_exp_groupings(exp_dir, subfolders):
  groupings_list = []
  for subfolder in subfolders:
    for exp in os.listdir(os.path.join(exp_dir, subfolder)):
      exp_path = os.path.join(exp_dir, subfolder, exp)
      groupings_list.append([exp_path, exp])
      # groupings_list.append([exp_path, exp.rsplit(' ')[-1][1:2]])  # original
  return pd.DataFrame(groupings_list, columns=['path', 'group'])

dir_name = '/content/drive/MyDrive/0 Flybowl/48 hrs-JAABAData'
folders = ['13.17.23', '13.25.49', '13.33.55', '13.42.27']
paths = get_exp_groupings(dir_name, folders)

# dir_name = '/content/drive/MyDrive/0 Flybowl/2025 FlyBowl JABBA data'
# folders = ['2-5-2025-30207JAABAData', '2-12-2025-30207JAABAData']
# paths = get_exp_groupings(dir_name, folders)

# folders_102864 = ['2-20-2025-102864JAABAData', '3-4-2025-102864JAABAData', '3-26-2025-102864JAABAData']
# paths_102864 = get_exp_groupings(dir_name, folders_102864)

# Extract and Bar Plot Data

In [75]:
def get_data(flybowls, beh, exp_time_secs, fps=30):
  max_length = 0
  all_data = None
  for flybowl in flybowls:
    t0 = sp.io.loadmat(os.path.join(flybowl, beh))['allScores'][0][0][5][0]
    t1 = sp.io.loadmat(os.path.join(flybowl, beh))['allScores'][0][0][6][0]

    n_flies = t0.shape[0]
    flybowl_data = np.zeros((n_flies, exp_time_secs*30))
    for fly in range(n_flies):
      t0_i, t1_i = t0[fly][0], t1[fly][0]
      for i in range(len(t0_i)):
        flybowl_data[fly][t0_i[i]:t1_i[i]] = 1

    flybowl_data = np.sum(flybowl_data, axis=0)
    if all_data is None:
      all_data = flybowl_data
    else:
      all_data = np.vstack([all_data, flybowl_data])
  return all_data

def clean_data(uncleaned_data, slices, slice_size, fps=30):
  df = pd.DataFrame(np.transpose(uncleaned_data))

  bins = np.arange(0, slices)*fps*slice_size
  df['bins'] = pd.cut(df.index, bins, right=False)
  df = df.groupby('bins', observed=False).sum()[0:60]

  return df

def graph(data_dict, beh, color_groupings, slices, output_folder):
  means, sem = {}, {}
  for group in groups:
    means[group] = np.mean(data_dict[group].T, axis=0).reset_index(drop=True)
    sem[group] = sp.stats.sem(data_dict[group].T)

  fig, ax = plt.subplots()
  fig.set_size_inches(20, 10)

  for group in groups:
    ax.plot(means[group], color=color_groupings[group], alpha=0.666, label=group)
    ax.fill_between(np.arange(0, len(means[group])), means[group]-sem[group], means[group]+sem[group], color=color_groupings[group], alpha=.2)

  leg = ax.legend(fontsize=12, frameon=False)
  for legobj in leg.get_lines():
      legobj.set_linewidth(3)

  ax.set_title(beh, fontsize=16)
  ax.set_xlabel('Slices', fontsize=14)
  ax.set_ylabel('Occurrences', fontsize=14)

  ax.set_xticks(np.arange(0, slices))
  ax.set_xticklabels(np.arange(slices), fontsize=12)
  ax.tick_params(axis='y', labelsize=12)

  ax.spines['top'].set_visible(False)
  ax.spines['right'].set_visible(False)

  plt.savefig(os.path.join(output_folder, f'{beh}.pdf'))

In [ ]:
time_secs, total_slices, slice_size_secs = 120, 12, 10
colors = {'Q1':'tab:blue', 'Q2':'tab:orange', 'Q3':'tab:green', 'Q4':'tab:red'}
groups = paths['group'].unique()
for beh in behaviors_list:
  beh_name = beh[beh.find("_")+1:beh.rfind(".")]

  data = {}
  for group in groups:
    data[group] = clean_data(get_data(paths[paths['group'] == group]['path'], beh, exp_time_secs=time_secs), slices=total_slices, slice_size=slice_size_secs)
    data[group].to_csv(f'/content/drive/MyDrive/0 Flybowl/test/{beh_name}/{group}_data.csv')

  graph(data, beh_name, colors, total_slices, '/content/drive/MyDrive/0 Flybowl/test')